In [1]:
import numpy as np
import pandas as pd
import re, json, csv, requests, time, glob, tqdm, datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from collections import Counter

In [9]:
def scrape_one_new(html, url):
    soup = BeautifulSoup(html, "html.parser")
    json_data = json.loads(soup.find_all("script", type="application/ld+json")[-1].text)
    title = json_data.get('headline', soup.find('span', class_='contentTitle').text)
    date = json_data.get('datePublished', re.search(r'datetime:.*?(\d{4}-\d{2}-\d{2}T\d{2}:\d{2})', str(html)).group(1))
    date_m = json_data.get('dateModified', '')
    genre = json_data.get('genre', [])
    if genre == []:
        genre = [k for k in soup.find('meta', attrs={'name':'keywords'}).get('content').split(',') if k not in ['NHK','ニュース', 'NHK NEWS WEB']]
    keywords = json_data.get('keywords', [])
    article = soup.find('div', id="news_textbody").text
    if soup.find_all('div', id="news_textmore") != []:
        for textmore in soup.find_all('div', id="news_textmore"):
            article += ('\n' + textmore.text)
    if soup.find_all('div', class_="news_add") != []:
        for newsadd in soup.find_all('div', class_="news_add"):
            if newsadd.h3 != None:
                newsadd.h3.extract()
            article += ('\n' + newsadd.text)
            
    return {
        'id':url.split('/')[-1].split('.html')[0],
        'title':title,
        'article':article.strip(),
        'genre':genre,
        'keywords':keywords,
        'url':url,
        'datePublished':date,
        'dateModified':date_m
    }

# for old web normal
def make_date_normal_old(ymd,time):
    year, month, day = ymd[:4], ymd[4:6], ymd[6:]
    hour, minute = time.split('時')
    minute = minute.strip('分')
    if len(hour) == 1:
        hour = '0' + hour
    if len(minute) == 1:
        minute = '0' + minute
    return f"{year}-{month}-{day}T{hour}:{minute}"

def scrape_one_old(html, url_true):
    soup = BeautifulSoup(html, "html.parser")
    title = soup.find('span', class_="contentTitle").text.strip()
    ymd_ = url_true.split('/')[-2]
    time_ = soup.find('span', id="news_time").text
    date = make_date_normal_old(ymd_, time_)
    genre = [k for k in soup.find('meta', attrs={'name':'keywords'}).get('content').split(',') if k not in ['NHK','ニュース', 'NHK NEWS WEB','ＮＨＫ','ＮＨＫニュース','']]
    article = soup.find(['div','p'], id="news_textbody").text
    if soup.find_all(['div','p'], id="news_textmore") != []:
        for textmore in soup.find_all(['div','p'], id="news_textmore"):
            article += ('\n' + textmore.text)
    if soup.find_all(['div','p'], class_="news_add") != []:
        for newsadd in soup.find_all(['div','p'], class_="news_add"):
            if newsadd.h3 != None:
                newsadd.h3.extract()
            article += ('\n' + newsadd.text)
            
    return {
        'id':url_true.split('/')[-1].split('.html')[0],
        'title':title,
        'article':article.strip(),
        'genre':genre,
        'keywords':[],
        'url':url_true,
        'datePublished':date,
        'dateModified':""
    }

def write_nolink(ID, year):
    with open(f'nolinknormal{year}.txt', 'a') as f:
        f.write(str(ID) + '\n')

def get_archiveurl_from_id(ID, date, http=True):
    if http:
        url = f'https://web.archive.org/web/*/http://www3.nhk.or.jp/news/html/{date}/k{ID}1000.html'
    else:
        url = f'https://web.archive.org/web/*/https://www3.nhk.or.jp/news/html/{date}/k{ID}1000.html'
    
    driver.get(url)
    time.sleep(5)
    html = str(driver.page_source.encode('utf-8'))
    snap = re.search(r'(times between|1 time|times).*?<a href="(.+?)">', html)
    archiveurl = 'https://web.archive.org' + snap.group(2)
    return None if 'nhk' not in archiveurl else archiveurl

def get_article_from_archiveurl(archiveurl, browser=True):
    if browser == False:
        response = requests.get(archiveurl)
        html = response.text
    else:
        driver.get(archiveurl)
        time.sleep(10)
        html = driver.page_source.encode('utf-8')

    url_true = 'http' + archiveurl.split('/http')[-1]
    if 'This page is not available on the web' in str(html):
        return None
    try:
        try:
            dic = scrape_one_old(html, url_true)
        except:
            dic = scrape_one_new(html, url_true)
        return dic
    except:
        return None

def js(dic, year):
    if dic == None:
        return
    with open(f'nhkweb{year}.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    with open(f'nhkweb{year}.json', 'w', encoding='utf-8') as f:
        if dic['id'] not in [x['id'] for x in data]:
            data.append(dic)
        else:
            for i, d in enumerate(data):
                if dic['id'] == d['id']:
                    data[i] = dic
        data = sorted(data, key=lambda x:x['id'])
        json.dump(data, f, indent=4, ensure_ascii=False)

def datebefore(ymd:str):
    year, month, day = int(ymd[:4]), int(ymd[4:6]), int(ymd[6:])
    before = str(datetime.datetime(year, month, day) + datetime.timedelta(-1))
    year, month, day = before[:4], before[5:7], before[8:10]
    return f'{year}{month}{day}'

def dateafter(ymd:str):
    year, month, day = int(ymd[:4]), int(ymd[4:6]), int(ymd[6:])
    before = str(datetime.datetime(year, month, day) + datetime.timedelta(1))
    year, month, day = before[:4], before[5:7], before[8:10]
    return f'{year}{month}{day}'

# check data

In [69]:
len(pd.read_json('nhkweb2019.json'))

42357

# get missing url & article

In [22]:
data = pd.read_json('nhkweb2018.json')
data = data[['id', 'url']]
data.id = data.id.apply(lambda x:x[1:-4])
data['date'] = data.url.apply(lambda x:x.split('news/html/')[-1].split('/')[0])
print(len(data))
print(data.head())
data[data.id == "1001132233"]

15990
           id                                                url      date
0  1001127630  http://www3.nhk.or.jp/news/html/20180101/k1001...  20180101
1  1001127631  http://www3.nhk.or.jp/news/html/20180101/k1001...  20180101
2  1001127632  http://www3.nhk.or.jp/news/html/20171231/k1001...  20171231
3  1001127633  http://www3.nhk.or.jp/news/html/20171231/k1001...  20171231
4  1001127634  http://www3.nhk.or.jp/news/html/20171231/k1001...  20171231


,id,url,date
4454,1001132233,http://www3.nhk.or.jp/news/html/20180209/k1001...,20180209


In [5]:
options = Options()
#options.headless = True
driver = webdriver.Chrome(options=options)

In [23]:
year = 2018

for i in tqdm.tqdm(range(4454, 10000)):
    # get the present row & next row
    ID1, ID2 = data.iat[i,0], data.iat[i+1,0]
    date = data.iat[i,2]
    before = datebefore(date)
    after = dateafter(date)

    # ID is continuous = no missing
    if int(ID1) + 1 == int(ID2):
        continue

    # not continuous
    else:  
        for ID in range(int(ID1)+1, int(ID2)):
            archiveurl = get_archiveurl_from_id(ID, date) # try the same day
            if archiveurl != None:
                dic = get_article_from_archiveurl(archiveurl)
                if dic:
                    js(dic, year)
                else:
                    write_nolink(ID, year) # error in NHK
            else:
                archiveurl = get_archiveurl_from_id(ID, after) # try the next day
                if archiveurl != None:
                    dic = get_article_from_archiveurl(archiveurl)
                    if dic:
                        js(dic, year)
                    else:
                        write_nolink(ID, year) # error in NHK
                else: 
                    archiveurl = get_archiveurl_from_id(ID, before) # try the previous day
                    if archiveurl != None:
                        dic = get_article_from_archiveurl(archiveurl)
                        if dic:
                            js(dic, year)
                        else:
                            write_nolink(ID, year) # error in NHK
                    else:
                        write_nolink(ID, year) # nolink







  0%|          | 0/5546 [00:00<?, ?it/s]




  0%|          | 7/5546 [00:47<10:26:08,  6.78s/it]




  0%|          | 10/5546 [01:44<16:07:54, 10.49s/it]




  1%|          | 41/5546 [02:06<11:33:11,  7.56s/it]




  1%|          | 42/5546 [02:26<17:12:32, 11.26s/it]




  1%|          | 45/5546 [02:45<14:57:54,  9.79s/it]




  1%|          | 62/5546 [03:05<10:59:03,  7.21s/it]




  1%|▏         | 71/5546 [03:24<8:36:39,  5.66s/it]




  1%|▏         | 74/5546 [04:22<14:51:55,  9.78s/it]




  1%|▏         | 78/5546 [04:40<12:27:59,  8.21s/it]




  2%|▏         | 84/5546 [05:21<11:49:52,  7.80s/it]




  2%|▏         | 86/5546 [05:53<15:36:25, 10.29s/it]




  3%|▎         | 140/5546 [06:43<11:13:47,  7.48s/it]




  3%|▎         | 145/5546 [07:01<9:28:46,  6.32s/it]




  3%|▎         | 152/5546 [07:19<7:45:59,  5.18s/it]




  3%|▎         | 162/5546 [07:59<7:12:37,  4.82s/it]




  3%|▎         | 164/5546 [08:52<16:57:42, 11.35s/it]




  3%|▎         | 169/5546 [09:28<15:05

AttributeError: 'NoneType' object has no attribute 'text'

In [20]:
ID1

'1001132231'

In [5]:
# delete fduplicate

nolinklist = pd.read_csv('nolinknormal.txt', header=None)
pd.DataFrame(sorted(set(map(str, nolinklist[0])) - set(data.id)))[0].to_csv('nolinknormal.txt', index=None)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [83]:
archiveurl = get_archiveurl_from_id(1001138686, 20180331)
archiveurl

'https://web.archive.org/web/20180401030106/http://www3.nhk.or.jp/news/html/20180331/k10011386861000.html'

In [85]:
response = requests.get(archiveurl)
response.text

''

In [41]:
scrape_one_old(html, archiveurl)

AttributeError: 'NoneType' object has no attribute 'text'

In [63]:
archiveurl = 'https://web.archive.org/web/20180104074901/http://www3.nhk.or.jp/news/html/20180104/k10011278991000.html'
driver.get(archiveurl)


{'id': 'k10011278991000',
 'title': '「北朝鮮は脅威」米でも８割 日米世論調査',
 'article': 'アメリカでトランプ政権が発足してから今月で１年を迎えるのを前に、ＮＨＫが日本とアメリカで実施した世論調査で、北朝鮮の核やミサイルの問題について「脅威だ」と答えた人がアメリカでも８割に上り、北朝鮮の脅威を現実のものとして国民が受け止めていることが明らかになりました。\nＮＨＫは、去年１１月２７日から１２月３日にかけて、日本とアメリカの１８歳以上の男女を対象に、コンピューターで無作為に発生させた固定電話と携帯電話の番号に電話をかける「ＲＤＤ」という方法で世論調査を行いました。調査では、日本で１２３２人、アメリカでは１２０１人から回答を得ました。\n\n\nＮＨＫは、去年１１月２７日から１２月３日にかけて、日本とアメリカの１８歳以上の男女を対象に、コンピューターで無作為に発生させた固定電話と携帯電話の番号に電話をかける「ＲＤＤ」という方法で世論調査を行いました。調査では、日本で１２３２人、アメリカでは１２０１人から回答を得ました。\n\n\n\n\nこの中で、北朝鮮の核やミサイルの問題はどの程度の脅威であると思うか聞いたところ、日本では「非常に脅威だ」が４８％、「ある程度脅威だ」が３３％と、脅威と感じる人は合わせて８１％でした。一方、アメリカでは「非常に脅威だ」が５０％、「ある程度脅威だ」が３３％と脅威と感じる人が合わせて８３％で、北朝鮮がアメリカ本土全域を攻撃できるＩＣＢＭ＝大陸間弾道ミサイルの発射実験に成功したと主張する中、アメリカでも国民が北朝鮮の脅威を現実のものとして受け止めていることが明らかになりました。\n\n\n\n\nさらに、北朝鮮の核やミサイルの問題を解決するためには何が最も有効だと思うか聞いたところ、アメリカでは、「話し合い」が最も多く３６％、次いで、「経済的圧力」が２４％、「軍事行動」が１８％、「軍事的圧力」が１７％という結果になりました。一方、日本では、最も多かったのが「経済的圧力」で３５％、次いで、「話し合い」が３１％、「軍事的圧力」が１５％、「軍事行動」が８％となっています。日本では、北朝鮮を脅威だと感じる人ほど「話し合い」による解決が有効だと考える人が増えるのに対し、アメリカでは逆に「話し合い」が減

In [77]:
html = driver.page_source.encode('utf-8')
scrape_one_old(html, archiveurl)

{'id': 'k10011276871000',
 'title': 'トランプ政権 北朝鮮対応「外交の時間限られている」',
 'article': 'アメリカのトランプ政権は、北朝鮮が早ければことしアメリカ本土に届く核ミサイルを実戦配備する可能性があるとして、中国やロシアをはじめ各国に外交による解決のための時間は限られていると訴えて、さらなる圧力を迫る構えです。\n北朝鮮への対応をめぐりアメリカのトランプ政権は、制裁などで圧力を最大限まで高めて北朝鮮の行動を改めさせ、事態の打開を目指しています。北朝鮮が早ければことし、アメリカ本土に届く核ミサイルを実戦配備する可能性があるとする情報機関の分析も伝えられ、ホワイトハウスの安全保障担当のマクマスター大統領補佐官も先月、「時間は非常に少なくなっている」と話しています。このためトランプ政権としては、先月、採択された国連の安全保障理事会の新たな制裁決議の履行の徹底とともに、今月、カナダで関係国による外相会合を開き、一層圧力を強化する方策を協議する予定です。そのうえで北朝鮮が行動を改めれば対話に入ることも可能だとするとともに、韓国から提案のあった来月のオリンピック期間中の訓練の延期についても検討を進めると見られます。トランプ政権は、北朝鮮に核ミサイルを保有させないためには軍事的な選択肢も排除しない姿勢を示していて、制裁の効果と北朝鮮の出方を見極めるとともに、中国やロシアをはじめ各国に外交による解決のための時間は限られていると訴えてさらなる圧力を迫る構えです。\n\n\nアメリカのＣＩＡ＝中央情報局で北朝鮮情勢の分析を担当したシンクタンク、ＣＳＩＳ＝戦略国際問題研究所のテリー上席研究員は、トランプ政権の北朝鮮への対応について、「キム・ジョンウン（金正恩）政権に最大限の圧力をかけるアプローチは正しい」として、政策の方向性は正しいと評価しています。ただ、「トランプ大統領は、いくつかの戦術的なミスも犯した」とも述べ、北朝鮮の完全な壊滅に言及するなど、過激な表現を用いたことが、必要以上に緊張を高めることにつながったとして、メッセージの発信には慎重さも求められると指摘します。そして、北朝鮮情勢をめぐることしの注目点として、韓国でオリンピックとパラリンピックが予定される２月から３月を挙げ、「韓国は同時期に例年行われるアメリカとの合同軍事